# CUB200

In [ ]:
import os
import sys

import quanproto.datasets.config_parser as quan_dataloader
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from quanproto.eda import eda
from quanproto.utils.workspace import *
from quanproto.utils.workspace import DATASET_DIR

In [ ]:
config = {
    "dataset_dir": DATASET_DIR,
    "dataset" : "cub200",
}

figure_height = 5
figure_width = 8

## Create the Dataset

In [ ]:
dataset = quan_dataloader.get_dataset(config["dataset_dir"], config["dataset"])

if not dataset.has_splits():
    dataset.split_dataset(predefined=True)

test_info = dataset.test_info()

In [ ]:
log_dir = os.path.join(WORKSPACE_PATH, "experiments/cub200/all")
os.makedirs(log_dir, exist_ok=True)

sample_labels = np.array(list(dataset.sample_labels().values()))
sample_dir = dataset.sample_dir()

# Make all image statistics
sample_class_labels = np.array(list(dataset.sample_labels().values()))
sample_dir = dataset.sample_dir()

In [ ]:
# Class Statistics
overview = eda.class_statistics(sample_class_labels)
overview

In [ ]:
# Class Distribution
counts, labels = eda.class_histogram(sample_class_labels)
fig, ax = plt.subplots(figsize=(figure_width, figure_height))
ax.plot(labels, counts)
plt.show()

In [ ]:
# Image Histograms
counts, vals = eda.color_histogram(sample_dir)
norm_counts = counts / np.sum(counts, axis=1, keepdims=True)
_, ax = plt.subplots(figsize=(figure_width, figure_height))
colors = ["red", "green", "blue", "black"]
labels = ["Red", "Green", "Blue", "Exposure"]

for i in range(4):
    ax.plot(vals, norm_counts[i], color=colors[i], label=labels[i])

ax.set_title("Color Histograms")
ax.set_xlabel("Pixel Value")
ax.set_ylabel("Frequency")
ax.legend()
plt.show()

In [ ]:
# Image Statistics
statistics = eda.color_statistics(counts, vals)
statistics

In [ ]:
import pandas as pd

k = 0
original_or_sam = "original"
# show an image
image = os.path.join(dataset.test_dirs()["test"], test_info["paths"][k])
labels = dataset.class_names()[test_info["labels"][k][0]]
bbox = test_info["bboxes"][k]
seg_mask = os.path.join(dataset.test_dirs()["segmentations"], original_or_sam, test_info["masks"]["original"]["paths"][k][0])
partlocs = test_info["partlocs"][k]
partnames = dataset.part_names()
attributes = test_info["attributes"][k]
attributnames = dataset.attribute_names()

# show the image
fig, ax = plt.subplots(1, 3, figsize=(figure_width * 3, figure_height))
# ax[0].imshow(plt.imread(image))
# ax[0].set_title(f"{labels.split('.')[1].replace('_', ' ')}")
ax[0].imshow(plt.imread(image))
ax[1].imshow(plt.imread(image))
ax[2].imshow(plt.imread(seg_mask))
ax[2].set_title("Segmentation Mask")

# Create a table for part names and their coordinates
part_names_coords = []
for key, points in partlocs.items():
    part_names_coords.append([partnames[key], points[0], points[1]])

part_names_df = pd.DataFrame(part_names_coords, columns=["Part Name", "X Coordinate", "Y Coordinate"])
print(part_names_df)

for i, points in enumerate(partlocs.values()):
    ax[1].scatter(points[0], points[1], s=10, c='r', marker='o')
    ax[1].set_title("Part Locations")

for bbox in test_info["bboxes"][k]:
    x_min, y_min, x_max, y_max = bbox
    rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=1, edgecolor='r', facecolor='none')
    ax[0].add_patch(rect)
    ax[0].set_title("Bounding Box")

for ax_ in ax:
    ax_.axis('off')